In [ ]:
!pip install langchain sentence-transformers chroma llama-cpp-python langchain_community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA,LLMChain

In [ ]:
loader=PyPDFDirectoryLoader("/content/drive/MyDrive/chatbot/data_set")
docs=loader.load()

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks=text_splitter.split_documents(documents=docs)

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']="hf_yYwcRsBwaxTVKAOqBjMeWyGxFeSgDn"#LlDd"
embeddings=SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")


In [ ]:
!pip install chromadb

In [ ]:
vectorstore=Chroma.from_documents(documents=chunks,embedding=embeddings)

In [ ]:
retriever=vectorstore.as_retriever(search_kwargs={'k':5})
retriever.get_relevant_documents("who are in the risk to get heart disease")

<ipython-input-12-6609837195fb>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("who are in the risk to get heart disease")


[Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/chatbot/data_set/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/chatbot/data_set/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/drive/MyDrive/chatbot/data_set/healthyheart.pdf'}, page_content='factor greatly increases the chances of deve

In [ ]:
llm=LlamaCpp(
    model_path="/content/drive/MyDrive/chatbot/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens=2028,
    top_p=1,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/chatbot/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attent

In [ ]:
template="""
<|context|>
Assume you are a medical assistant bot.
Your role is to provide accurate and concise information about medical conditions, treatments, and health advice.
Explain complex medical concepts in simple terms and be truthful.
previous conversations:
{history}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain=(
      {"context":retriever,"query":RunnablePassthrough()}
      |prompt
      |llm
      |StrOutputParser()
)

In [ ]:
while True:
  query=input("Enter your query:")
  if query=="exit":
    break
  elif query=="":
    continue
  else:
    response=rag_chain.invoke(query)
    print(response)

Enter your query:exit
